# Differential expression testing human ICM
For the human ICM data, we are interesting in one primary test, differences in ICM vs NF by cell type. To perform differential expression testing, we will use the summation + limma-voom approach, as proposed by Lun and Marioni (https://academic.oup.com/biostatistics/article/18/3/451/2970368?login=true) and shown to be slightly conservative, if anything, based on Zimmerman, Espeland, and Langefeld (https://www.nature.com/articles/s41467-021-21038-1).

The model we will employ is `expr ~ 1 + disease + sex + age`

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

In [2]:
# Libraries
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import os
import scanpy as sc

# Using rpy2 to call R code from python
import rpy2
import rpy2.situation as rsetup
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.conversion import localconverter

import warnings

/home/unix/chaffin/.local/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:15: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


In [3]:
# R packages required
base = importr('base')
edgeR = importr('edgeR')
DESeq2 = importr('DESeq2')
limma = importr('limma')
stats = importr('stats')
grdevices = importr('grDevices')

In [4]:
# Python package versions
print('scanpy: ' + sc.__version__)
print('pandas: ' + pd.__version__)
print('numpy: ' + np.__version__)
print('rpy2: ' + rpy2.__version__)

scanpy: 1.7.2
pandas: 1.2.4
numpy: 1.18.1
rpy2: 3.0.5


In [5]:
# R package version
print('R version: ' + base.__version__)
print('edgeR: ' + edgeR.__version__)
print('DESeq2: ' + DESeq2.__version__)
print('limma: ' + limma.__version__)
print('stats: ' + stats.__version__)
print('grdevices: ' + grdevices.__version__)

R version: 3.6.0
edgeR: 3.30.3
DESeq2: 1.20.0
limma: 3.44.3
stats: 3.6.0
grdevices: 3.6.0


In [6]:
# Turn off some annoying FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Set path for aggregation data and output data

In [7]:
path_aggregation = '../../data/aggregation'
path_diffexpress = '../../results/diffexpress'

In [ ]:
# create the folder to store results in if it doesn't exist
if not os.path.exists(path_diffexpress):
    os.mkdir(path_diffexpress)

## Comparison: ICM vs NF
For each cell type, we want to test for DE between ICM and NF across all genes. We will only sum counts across all nuclei of a given type in a sample if the sample has more than 25 nuclei of the type. Otherwise the sample will be excluded from the differential expression test. As a sensitivity check, and to ensure our results are robust, we will test for DE using both the CellRanger counts and the CellBender remove-background adjusted counts.

We will filter out genes in < 1% of nuclei from BOTH the ICM and NF groups as these are too lowly expressed to test, as well as an additional filter of `filterByExpr` as implemented in edgeR.

In [10]:
# Read in the snRNAseq map
adata = sc.read(os.path.join(path_aggregation,'humanICM.PostQC.Map/PostQC.Neighbor.UMAP.Freeze1.h5ad'))

Only considering the two last: ['.Freeze1', '.h5ad'].
Only considering the two last: ['.Freeze1', '.h5ad'].


In [16]:
# Inspect the number of samples that will be included in a DE test for each cell type
base = (adata.obs[['individual','disease',
                   'leiden0.5','sample']].groupby(['individual','disease','leiden0.5']).count()).reset_index()
base['good'] = base['sample'] > 25
base[['leiden0.5','disease','good']].groupby(['leiden0.5','disease']).sum().reset_index().pivot(index='leiden0.5',
                                                                                                columns='disease',
                                                                                                values='good')

disease,ICM,NF
leiden0.5,,
0,7,8
1,7,8
2,7,8
3,7,8
4,7,8
5,7,8
6,7,8
7,7,8
8,7,7


We will only be able to test for differential expression for clusters 0 - 10, as the remaining clusters only have enough nuclei from one condition. We will have to be cautious with cluster 9 (adipocytes) given only 2 samples are used from the NF group.

In [17]:
# Group the leiden clusters into the cell types we want to test
groups = {'pseudo_bulk':[str(x) for x in range(len(adata.obs['leiden0.5'].unique()))], # this is ALL clusters combined
          'cardiomyocyte':['0'],
          'fibroblast':['1'],
          'endothelial1':['2'],
          'pericyte':['3'],
          'macrophage':['4'],
          'endothelial2':['5'],
          'lymphocyte':['6'],
          'vsmc':['7'],
          'lymphatic_endothelial':['8'],
          'adipocyte':['9'],
          'neuronal':['10']}

In [18]:
'''
Function to perform differential expression test
'''
def run_differential_expression(bdata, aggr_var, min_cell, group_var, target_group, ref_group,
                                covar, covar_dtype, cellranger, out):
    '''
    bdata: the subset of nuclei to include in the test
    aggr_var: the .obs key to sum counts across (e.g., individual)
    min_cell: the minimum cells to consider in a sample for aggregating
    group_var: the .obs key for the grouping variable
    target_group: the target group of the group_var to compare to ref_group
    ref_group: the reference group of the DE test
    covar: covariates to include in the DE test
    covar_dtype: the datatype for the covariates
    cellranger: True/False indicating whether to use .layers['cellranger_raw'] or not
    out: output file prefix
    '''
    # Aggregate the counts by the aggr_var
    # This generates the "pseudo-bulk" count matrix
    tobind = [] # fill in each aggregation group, column-wise
    if cellranger:
        cellranger_key = 'cellranger'
        for ind in bdata.obs[aggr_var].unique():
            bdata_subset = bdata[bdata.obs[aggr_var]==ind].copy()
            if bdata_subset.shape[0] > min_cell:
                append_vect = pd.DataFrame(bdata_subset.layers['cellranger_raw'].sum(0)).transpose()
                append_vect.columns = [ind]
                tobind.append(append_vect) 
    else:
        cellranger_key = 'cellbender'
        for ind in bdata.obs[aggr_var].unique():
            bdata_subset = bdata[bdata.obs[aggr_var]==ind].copy()
            if bdata_subset.shape[0] > min_cell:
                append_vect = pd.DataFrame(bdata_subset.X.sum(0)).transpose()
                append_vect.columns = [ind]
                tobind.append(append_vect)
    summed_exp = pd.concat(tobind, axis = 1)
    summed_exp.index = bdata.var.index
    # Only keep samples that are in the groups being compared
    keep_samp = bdata.obs[bdata.obs[group_var].isin([target_group, ref_group])][aggr_var].unique()
    summed_exp = summed_exp[[x for x in keep_samp.tolist() if x in summed_exp.columns]]
    
    # Filter out lowly expressed genes -- this is always done on CellBender counts
    # so that a similar set of genes goes into the DE test (even if using the CellRanger counts for testing)
    # This removes genes in < 1% of both comparison groups
    keep_gene = bdata.var[(np.array((bdata[bdata.obs[group_var] == target_group].X > 0).mean(0) > 0.01).squeeze())|
                          (np.array((bdata[bdata.obs[group_var] == ref_group].X > 0).mean(0) > 0.01).squeeze())].index.tolist()
    summed_exp = summed_exp[summed_exp.index.isin(keep_gene)]
    print('\tGenes with > 1% in either group = ' + str(summed_exp.shape[0]))

    # Set up data in R
    meta_data = bdata.obs[[aggr_var, group_var] + covar].drop_duplicates().reset_index().drop('index',1)
    meta_data[aggr_var] = ['X' + str(x) if x[0].isdigit() else str(x) for x in meta_data[aggr_var]]

    with localconverter(ro.default_converter + pandas2ri.converter):
        all_counts = ro.conversion.py2rpy(summed_exp) # count data
    with localconverter(ro.default_converter + pandas2ri.converter): 
        meta_data_r = ro.conversion.py2rpy(meta_data) # meta data
    ro.r.assign('all_counts', all_counts)
    ro.r.assign('meta_data_r', meta_data_r)
    
    # Remove MT and RB genes
    ro.r('mt_genes <- rownames(all_counts)[regexpr("MT-",rownames(all_counts)) != -1]')
    ro.r('rb_genes <- rownames(all_counts)[regexpr("^RP[SL]",rownames(all_counts)) != -1]')
    ro.r('all_counts <- all_counts[which(!rownames(all_counts) %in% mt_genes & !rownames(all_counts) %in% rb_genes),]')
    
    # Set up grouping variable and covariates
    ro.r('group <- with(meta_data_r, ' + group_var + '[match(colnames(all_counts), ' + aggr_var + ')])')
    for cov, covt in zip(covar, covar_dtype):
        if covt == 'factor':
            ro.r('as.factor(' + cov + ' <- with(meta_data_r, ' + cov + '[match(colnames(all_counts), ' + aggr_var + ')]))')
        elif covt == 'numeric':
            ro.r('as.numeric(' + cov + ' <- with(meta_data_r, ' + cov + '[match(colnames(all_counts), ' + aggr_var + ')]))')    

    # filter out genes with low expression -- we'll use the EdgeR function to determine what "low" is
    ro.r('keep.exprs <- filterByExpr(all_counts, group = group)')
    ro.r('all_counts <- all_counts[keep.exprs, ]')
    print(ro.r('paste0("      Genes retained after filterByExpr = ",dim(all_counts)[1])'))
    print(ro.r('table(group)'))
    
    # Set up for limma-voom
    ro.r('sample.formula <- ~ 1+group+' + '+'.join(covar))
    ro.r('sample.data <- data.frame(group=factor(group, levels=c("' + ref_group + '","' + target_group + '")),' + ','.join(covar) + ')')
    ro.r('design <- model.matrix(sample.formula, sample.data)')
    # Normalize data
    sf = DESeq2.estimateSizeFactorsForMatrix(ro.r('all_counts')) # DESeq2 normalization
    ro.r.assign('sf', sf)
    ro.r('eff.lib <- sf*mean(colSums(all_counts))')
    print('\tdata normalized...')
    
    # Run Voom
    ro.r('y <- DGEList(all_counts, lib.size=eff.lib)')
    grdevices.pdf(file=out + '_' + cellranger_key + '_voom.pdf')
    ro.r('v.all <- voomWithQualityWeights(y, design, plot=T)')
    grdevices.dev_off()
    
    ro.r('vfit <- lmFit(v.all, design)')
    ro.r('vfit <- eBayes(vfit, robust=TRUE)')
    ro.r('vres <- topTable(vfit, n=Inf, sort.by="P", coef="group' + target_group +'")')
    ro.r('write.table(vres,"' + out + '_' + cellranger_key + '_results.txt", , sep="\t", quote=F)')

    # Extra Voom diagnostic (plotSA)
    grdevices.pdf(file=out + '_' + cellranger_key + '_plotSA.pdf')
    ro.r('plotSA(vfit)')
    grdevices.dev_off()
    print('Done!')

In [19]:
# Make the folder to store these DE results
if not os.path.exists(os.path.join(path_diffexpress, 'human_ICM_vs_NF')):
    os.mkdir(os.path.join(path_diffexpress, 'human_ICM_vs_NF'))

In [20]:
# loop through each cell type and run the DE test
# Once on cellbender counts and once on cellranger counts
for celltype in groups.keys():
    print('Testing celltype (CellBender) ' + celltype)
    run_differential_expression(bdata = adata[adata.obs['leiden0.5'].isin(groups[celltype])].copy(),
                                aggr_var = 'individual',
                                min_cell = 25,
                                group_var = 'disease',
                                target_group = 'ICM',
                                ref_group = 'NF',
                                covar = ['sex', 'age'],
                                covar_dtype = ['factor', 'numeric'],
                                cellranger = False,
                                out = os.path.join(path_diffexpress, 'human_ICM_vs_NF/' + celltype))
    print('Testing celltype (CellRanger) ' + celltype)
    run_differential_expression(bdata = adata[adata.obs['leiden0.5'].isin(groups[celltype])].copy(),
                                aggr_var = 'individual',
                                min_cell = 25,
                                group_var = 'disease',
                                target_group = 'ICM',
                                ref_group = 'NF',
                                covar = ['sex', 'age'],
                                covar_dtype = ['factor', 'numeric'],
                                cellranger = True,
                                out = os.path.join(path_diffexpress, 'human_ICM_vs_NF/' + celltype))    

Testing celltype (CellBender) pseudo_bulk
	Genes with > 1% in either group = 12507
[1] "      Genes retained after filterByExpr = 12449"

group
ICM  NF 
  7   8 

	data normalized...
Done!
Testing celltype (CellRanger) pseudo_bulk
	Genes with > 1% in either group = 12507
[1] "      Genes retained after filterByExpr = 12450"

group
ICM  NF 
  7   8 

	data normalized...
Done!
Testing celltype (CellBender) cardiomyocyte
	Genes with > 1% in either group = 13467
[1] "      Genes retained after filterByExpr = 13236"

group
ICM  NF 
  7   8 

	data normalized...
Done!
Testing celltype (CellRanger) cardiomyocyte
	Genes with > 1% in either group = 13467
[1] "      Genes retained after filterByExpr = 13381"

group
ICM  NF 
  7   8 

	data normalized...
Done!
Testing celltype (CellBender) fibroblast
	Genes with > 1% in either group = 10704
[1] "      Genes retained after filterByExpr = 10267"

group
ICM  NF 
  7   8 

	data normalized...
Done!
Testing celltype (CellRanger) fibroblast
	Genes with

### Combine all DE results into one results file and annotate additional information
We want to aggregate all the information across all cell types into one file, and append in some additional information including:
1. Pct > 0 in both ICM and NF groups
2. Avg Expr in both ICM and NF groups
3. Heuristic metric indiciating if the gene expression for a given gene is likely to come from background

In [21]:
'''
function to calculate the background probability heuristic
this is a metric that provides a sense of whether the gene
expression is really driven by background contamination as
CellBender is imperfect
'''
def calculate_bkg(adata, cluster_key, cluster):
    # Approximate background profile across full dataset
    # Based on cumulative distribution function
    # Intuition: If a gene is not highly expressed in the entire experiment,
    # it's not likely to be highly expressed in background
    approx_background_profile = np.array(adata.X.sum(axis=0)).squeeze() / adata.X.sum()  # normalized
    gene_order = np.argsort(approx_background_profile)
    gene_to_cdf_x = np.argsort(gene_order)
    cdf = np.cumsum(approx_background_profile[gene_order])
    adata.var['gene_bkg_prob'] = cdf[[gene_to_cdf_x[i] for i in range(len(adata.var))]]
    
    # counts for inside and outside the target cluster
    adata_subset = adata[adata.obs[cluster_key].isin(cluster)]
    bc_out_group = list(set(adata.obs.index) - set(adata_subset.obs.index))
    adata_out_group = adata[bc_out_group]
    X_in = adata_subset.X
    X_out = adata_out_group.X

    # calculate fraction of cells in and out of group expressing > 0 counts
    tp_0 = np.array((X_in > 0).sum(axis=0)).squeeze()
    fp_0 = np.array((X_out > 0).sum(axis=0)).squeeze()
    frac_exp_gr_0_target = tp_0 / adata_subset.shape[0]
    frac_exp_gr_0_other = fp_0 / X_out.shape[0]

    # calculate fraction of cells in and out of group expressing > 1 counts
    # This can be helpful because there are often a lot of 1 counts for genes
    tp_1 = np.array((X_in > 1).sum(axis=0)).squeeze()
    fp_1 = np.array((X_out > 1).sum(axis=0)).squeeze()
    frac_exp_gr_1_target = tp_1 / adata_subset.shape[0]
    frac_exp_gr_1_other = fp_1 / X_out.shape[0]

    # calculate standardized PPVs based on > 0 expression and > 1 expression
    # this standardized PPV is equivalent to what would be calculated from
    # specificity and sensivity with a set prevalence of 0.5 (e.g. assuming equal cluster size)
    # Note: a small value is added to the denominator to avoid division by 0
    ppv_0 = frac_exp_gr_0_target / (frac_exp_gr_0_target + frac_exp_gr_0_other + 1e-5)
    ppv_1 = frac_exp_gr_1_target / (frac_exp_gr_1_target + frac_exp_gr_1_other + 1e-5)
    
    # take the average ppv0 and ppv1
    mean_ppv = (ppv_0 + ppv_1) / 2.
    prob_from_another_cluster = 1. - mean_ppv  # if very low mean_ppv, unlikely from this given cluster
    bkg_prob = prob_from_another_cluster * adata.var['gene_bkg_prob'] # normalize this for the overall expression of the gene
    return(bkg_prob)

In [22]:
# Read in the snRNAseq map
adata = sc.read(os.path.join(path_aggregation,'humanICM.PostQC.Map/PostQC.Neighbor.UMAP.Freeze1.h5ad'))

Only considering the two last: ['.Freeze1', '.h5ad'].
Only considering the two last: ['.Freeze1', '.h5ad'].


In [23]:
# loop through each cell type and compile the information required
target_group = 'ICM'
ref_group = 'NF'
group_var = 'disease'
min_cell = 25 # matches above minimum cells required to aggregate
tocat = []
for gr in groups.keys():
    print(gr)
    # get the DE results
    de_cb = pd.read_csv(os.path.join(path_diffexpress, 'human_ICM_vs_NF/' + gr + '_cellbender_results.txt'),
                        sep='\t').reset_index() # cellbender DE
    de_cb.columns = ['index'] + [x + '_CB' for x in de_cb.columns[1:]] # rename columns
    de_cr = pd.read_csv(os.path.join(path_diffexpress, 'human_ICM_vs_NF/' + gr + '_cellranger_results.txt'),
                        sep='\t').reset_index() # cellranger DE
    de_cr.columns = ['index'] + [x + '_CR' for x in de_cr.columns[1:]] # rename columns
    de = de_cb.merge(de_cr, on = 'index', how = 'left')
    de = de.merge(adata.var.reset_index()[['index', 'gene_ids']], on = 'index', how = 'left')
    # Add in the test-level annotation like number of cells/samples in the test per group
    de['celltype'] = gr
    de['grp1'] = target_group
    de['grp2'] = ref_group
    de['ncell_grp1'] = adata.obs[(adata.obs[group_var] == target_group)&(adata.obs['leiden0.5'].isin(groups[gr]))].shape[0]
    de['ncell_grp2'] = adata.obs[(adata.obs[group_var] == ref_group)&(adata.obs['leiden0.5'].isin(groups[gr]))].shape[0]
    de['nsamp_grp1'] = (adata.obs[(adata.obs[group_var] == target_group)&
                                  (adata.obs['leiden0.5'].isin(groups[gr]))]['individual'].value_counts() > 25).sum()
    de['nsamp_grp2'] = (adata.obs[(adata.obs[group_var] == ref_group)&
                                  (adata.obs['leiden0.5'].isin(groups[gr]))]['individual'].value_counts() > 25).sum()
        
    # add the background prob, unless pseudo-bulk where not relevant
    if gr == 'pseudo_bulk':
        de['gene_bkg_prob'] = np.nan
    else:
        bkg_prob = pd.DataFrame(calculate_bkg(adata, 'leiden0.5', groups[gr])).reset_index()
        bkg_prob.columns = ['gene','gene_bkg_prob']
        de = de.merge(bkg_prob, left_on='index', right_on='gene', how='left')
        
    # pct and avg express for each gene
    adata_clst = adata[adata.obs['leiden0.5'].isin(groups[gr])].copy()
    sc.pp.normalize_total(adata_clst, target_sum=1e4)
    sc.pp.log1p(adata_clst)
    target_avg_expr = np.array(adata_clst[adata_clst.obs[group_var] == target_group].X.mean(0)).squeeze()
    ref_avg_expr = np.array(adata_clst[adata_clst.obs[group_var] == ref_group].X.mean(0)).squeeze()
    target_pct_gt0 = np.array((adata_clst[adata_clst.obs[group_var] == target_group].X > 0).mean(0)).squeeze()
    ref_pct_gt0 = np.array((adata_clst[adata_clst.obs[group_var] == ref_group].X > 0).mean(0)).squeeze()    
    expr_level = pd.DataFrame({'pctgt0_grp1':target_pct_gt0,
                               'pctgt0_grp2':ref_pct_gt0,
                               'avgexpr_grp1':target_avg_expr,
                               'avgexpr_grp2':ref_avg_expr})
    expr_level['index'] = adata.var.index.tolist()
    de = de.merge(expr_level, left_on='index', right_on='index', how='left')
    
    # reorganize the data
    de = de[['index', 'gene_ids', 'celltype', 'grp1', 'grp2', 'nsamp_grp1', 'nsamp_grp2',
             'ncell_grp1', 'ncell_grp2', 'pctgt0_grp1', 'pctgt0_grp2', 'avgexpr_grp1', 'avgexpr_grp2',
             'gene_bkg_prob',
             'logFC_CB', 'AveExpr_CB', 't_CB', 'P.Value_CB', 'adj.P.Val_CB',
             'logFC_CR', 'AveExpr_CR', 't_CR', 'P.Value_CR', 'adj.P.Val_CR']]
    de.columns = ['gene', 'ensembl_id', 'celltype', 'grp1', 'grp2', 'nsamp_grp1', 'nsamp_grp2',
                      'ncell_grp1', 'ncell_grp2', 'pctgt0_grp1', 'pctgt0_grp2', 'avgexpr_grp1', 'avgexpr_grp2',
                      'gene_bkg_prob', 'logFC_CB', 'AveExpr_CB', 't_CB', 'P.Value_CB', 'adj.P.Val_CB',
                     'logFC_CR', 'AveExpr_CR', 't_CR', 'P.Value_CR', 'adj.P.Val_CR']
    tocat.append(de)

# combine all the DE results across cell types together
allres = pd.concat(tocat)

pseudo_bulk
cardiomyocyte
fibroblast
endothelial1
pericyte
macrophage
endothelial2
lymphocyte
vsmc
lymphatic_endothelial
adipocyte
neuronal


In [24]:
allres.to_csv(os.path.join(path_diffexpress, 'human_ICM_vs_NF_differential_expression_v2.0.txt.gz'),
               sep='\t', compression='gzip', na_rep='NA', index=None)